# Web Scraping Toronto Neighbourhood Data

In [1]:
import requests
import pandas as pd
import csv

In [2]:
#!conda install beautifulsoup4
from bs4 import BeautifulSoup

# Web Scraping to acquire required neighbourhood Data

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')
file = open('postal_codes_toronto.csv', 'w')
csv_wr = csv.writer(file)
csv_wr.writerow(['PostalCode', 'Borough', 'Neighbourhood'])

table = soup.find('table', class_ = 'wikitable')
table_rows = table.find_all('tr')
postal_codes = []
boroughs = [] 
neighbourhoods = [] 

for row in table_rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned':  
            
            postalcode = columns[0].text.split('\n')[0]
            postal_codes.append(postalcode)
            borough = columns[1].text.split('\n')[0]
            boroughs.append(borough)
            neighbourhood = columns[2].text.split('\n')[0]    
            neighbourhoods.append(neighbourhood)
            
    except Exception as e : 
        pass    
    
    
postal_present = []
for index_i, postal_i in enumerate(postal_codes) :   
    if postal_i not in postal_present :
        nbds = neighbourhoods[index_i]
        for index_f, postal_f in enumerate(postal_codes) :
            if postal_i == postal_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_wr.writerow([postal_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postal_present.append(postal_i)
file.close()

# Creating a DataFrame

In [5]:
df = pd.read_csv('postal_codes_toronto.csv')

In [6]:
df.shape

(180, 3)

In [7]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [8]:
df1=df.dropna()

In [11]:
df1.reset_index(drop=True,inplace=True)

In [12]:
df1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


# Replacing / in Neighborhood with ,

In [13]:
df1['Neighbourhood'] = df1.Neighbourhood.str.replace('/',',')

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
df1 = df1.astype(str)

In [17]:
df1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [16]:
df1.shape

(103, 3)